# Wood density derived biomass ensemble calculation in GEE

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
from termcolor import colored # this is allocate colour and fonts type for the print title and text

In [2]:
# initialize the earth engine API
ee.Initialize()

## load all the biomass images into a multiband image

In [3]:
# load the composite where we can extract the GSV layer
growingStockVolume = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec").select('GlobBiomass_GrowingStockVolume');
# load the present forest cover 
presentForestCover = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec").select(['HansenEtAl_TreeCover_Year2010']).divide(100).unmask(); # uniform with potential in the  0-1 scale
# define the forest mask
forestMask = presentForestCover.gte(0.00);
# generete the pixel area map with unit ha
pixelArea = ee.Image.pixelArea().divide(10000);

In [4]:
# load the images for the 30 ensemble models
# define an empty image
firstImage = ee.Image('users/leonidmoore/WoodDensityProject/PredictedMaps/Predicted_WoodDensity_BufferBased_Mean_Map_with_Seed_0').rename('Model_0')

modelList = np.arange(1, 200, 1).tolist()
for ml in modelList:
    perModelImage = ee.Image('users/leonidmoore/WoodDensityProject/PredictedMaps/Predicted_WoodDensity_BufferBased_Mean_Map_with_Seed_'+str(ml)).rename('Model_'+str(ml))
    firstImage = firstImage.addBands(perModelImage)

print(colored('The band names are:', 'blue', attrs=['bold']),firstImage.bandNames().getInfo())

The band names are: ['Model_0', 'Model_1', 'Model_2', 'Model_3', 'Model_4', 'Model_5', 'Model_6', 'Model_7', 'Model_8', 'Model_9', 'Model_10', 'Model_11', 'Model_12', 'Model_13', 'Model_14', 'Model_15', 'Model_16', 'Model_17', 'Model_18', 'Model_19', 'Model_20', 'Model_21', 'Model_22', 'Model_23', 'Model_24', 'Model_25', 'Model_26', 'Model_27', 'Model_28', 'Model_29', 'Model_30', 'Model_31', 'Model_32', 'Model_33', 'Model_34', 'Model_35', 'Model_36', 'Model_37', 'Model_38', 'Model_39', 'Model_40', 'Model_41', 'Model_42', 'Model_43', 'Model_44', 'Model_45', 'Model_46', 'Model_47', 'Model_48', 'Model_49', 'Model_50', 'Model_51', 'Model_52', 'Model_53', 'Model_54', 'Model_55', 'Model_56', 'Model_57', 'Model_58', 'Model_59', 'Model_60', 'Model_61', 'Model_62', 'Model_63', 'Model_64', 'Model_65', 'Model_66', 'Model_67', 'Model_68', 'Model_69', 'Model_70', 'Model_71', 'Model_72', 'Model_73', 'Model_74', 'Model_75', 'Model_76', 'Model_77', 'Model_78', 'Model_79', 'Model_80', 'Model_81', 'Mode

In [5]:
# transfer the multiband image of wood density into biomass
stemBiomassImage = firstImage.multiply(growingStockVolume)
WWFBiome = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec").select('WWF_Biome')
# define masks by biome information
mask01 = WWFBiome.eq(1)
mask02 = WWFBiome.eq(2)
mask03 = WWFBiome.eq(3)
mask04 = WWFBiome.eq(4)
mask05 = WWFBiome.eq(5)
mask06 = WWFBiome.eq(6)
mask07 = WWFBiome.eq(7)
mask08 = WWFBiome.eq(8)
mask09 = WWFBiome.eq(9)
mask10 = WWFBiome.eq(10)
mask11= WWFBiome.eq(11)
mask12 = WWFBiome.eq(12)
mask13 = WWFBiome.eq(13)

# calculate the biomass density of above ground for each biome
densityBiome01 = stemBiomassImage.mask(mask01).multiply(1.39)
densityBiome02 = stemBiomassImage.mask(mask02).multiply(1.40)
densityBiome03 = stemBiomassImage.mask(mask03).multiply(1.33)
densityBiome04 = stemBiomassImage.mask(mask04).multiply(1.35)
densityBiome05 = stemBiomassImage.mask(mask05).multiply(1.37)
densityBiome06 = stemBiomassImage.mask(mask06).multiply(1.33)
densityBiome07 = stemBiomassImage.mask(mask07).multiply(2.25)
densityBiome08 = stemBiomassImage.mask(mask08).multiply(1.39)
densityBiome09 = stemBiomassImage.mask(mask09).multiply(1.26)
densityBiome10 = stemBiomassImage.mask(mask10).multiply(1.32)
densityBiome11 = stemBiomassImage.mask(mask11).multiply(1.42)
densityBiome12 = stemBiomassImage.mask(mask12).multiply(1.32)
densityBiome13 = stemBiomassImage.mask(mask13).multiply(1.31)
# compile into a collection
allBiomeDensityComposite = ee.ImageCollection.fromImages([densityBiome01,
                                                          densityBiome02,
                                                          densityBiome03,
                                                          densityBiome04,
                                                          densityBiome05,
                                                          densityBiome06,
                                                          densityBiome07,
                                                          densityBiome08,
                                                          densityBiome09,
                                                          densityBiome10,
                                                          densityBiome11,
                                                          densityBiome12,
                                                          densityBiome13])

In [6]:
# mosaic the biome level map into global ones
mergedAGB_Density = allBiomeDensityComposite.mosaic()
# get the AGB absolute value map
# absoluteAGB_Map = mergedAGB_Density.multiply(pixelArea).divide(1000000000)
# load the root shoot ratio map with the interpolated window map
rootShootRatio = ee.Image("users/leonidmoore/WoodDensityProject/Root_Shoot_Ratio_Map")
mergedTGB_Density = mergedAGB_Density.add(mergedAGB_Density.multiply(rootShootRatio))

In [7]:
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## Calculate the Mean and variation coefficient maps of wood density mean

In [8]:
# calculate the mean and variation images
meanImage = mergedTGB_Density.reduce(ee.Reducer.mean())
variImage = mergedTGB_Density.reduce(ee.Reducer.stdDev()).divide(meanImage)
#95% quantile range
quantileImage = mergedTGB_Density.reduce(ee.Reducer.percentile([2.5,97.5],['lower','upper']))

## Export Mean and variation coefficient maps of wood density mean to Asset

In [9]:
# add those two images into the GEE assets
meanExport = ee.batch.Export.image.toAsset(image = meanImage,
                                           description = '20220808_Biomass_Density_Mean_Ensemble_Map_Mean_To_Asset',
                                           assetId = 'users/leonidmoore/WoodDensityProject/DerivedBiomassMaps/WoodDensity_Derived_Biomass_Density_Ensemble_Mean_BufferBased',
                                           region = unboundedGeo,
                                           crs = 'EPSG:4326',
                                           crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                           maxPixels = 1e13)


# start the export task
meanExport.start()
# show the task status
meanExport.status()

variExport = ee.batch.Export.image.toAsset(image = variImage,
                                           description = '20220808_Biomass_Density_Mean_Ensemble_Map_Vari_To_Asset',
                                           assetId = 'users/leonidmoore/WoodDensityProject/DerivedBiomassMaps/WoodDensity_Derived_Biomass_Density_Ensemble_Vari_BufferBased',
                                           region = unboundedGeo,
                                           crs = 'EPSG:4326',
                                           crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                           maxPixels = 1e13)

# start the export task
variExport.start()
# show the task status
variExport.status()

{'state': 'READY',
 'description': '20220808_Biomass_Density_Mean_Ensemble_Map_Vari_To_Asset',
 'creation_timestamp_ms': 1663094449436,
 'update_timestamp_ms': 1663094449436,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'WQPJ4L6FNLORVEUQGFPCXR6C',
 'name': 'projects/earthengine-legacy/operations/WQPJ4L6FNLORVEUQGFPCXR6C'}

In [10]:
quantileExport = ee.batch.Export.image.toAsset(image = quantileImage,
                                           description = '20220808_Biomass_Density_Mean_Ensemble_Map_Quantile_To_Asset',
                                           assetId = 'users/leonidmoore/WoodDensityProject/DerivedBiomassMaps/WoodDensity_Derived_Biomass_Density_Ensemble_Quantile_BufferBased',
                                           region = unboundedGeo,
                                           crs = 'EPSG:4326',
                                           crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                           maxPixels = 1e13)

# start the export task
quantileExport.start()
# show the task status
quantileExport.status()

{'state': 'READY',
 'description': '20220808_Biomass_Density_Mean_Ensemble_Map_Quantile_To_Asset',
 'creation_timestamp_ms': 1663094450908,
 'update_timestamp_ms': 1663094450908,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'LFC7DKT46HN37SUVL566J764',
 'name': 'projects/earthengine-legacy/operations/LFC7DKT46HN37SUVL566J764'}